# Requirements and Libraries

In [16]:
pip install  lightgbm  lifelines 

Note: you may need to restart the kernel to use updated packages.


In [17]:
# aquí dejo un "special import" (luego tengo temas con este import)
def _pip_install(pkg):
    try:
        __import__(pkg.split('==')[0].replace("-","_"))
    except ImportError:
        import subprocess, sys
        subprocess.check_call([sys.executeable, "-m", "pip", "install", pkg])
        
for pkg in ['holidays']:
    _pip_install(pkg)
        

In [18]:
#basics
import os
import sys
import re
import json
import warnings
import joblib
import unicodedata
import logging

#data manipulation
import pandas as pd
import numpy as np
import typing as t
import holidays

#Data Viz
import seaborn as sns
import matplotlib.pyplot as plt

#info
from pathlib import Path
from zoneinfo import ZoneInfo
from dataclasses import dataclass
from datetime import datetime, date

#google bigquery
from google.cloud import bigquery

#statistics
import scipy
from scipy import stats
from scipy.stats import ks_2samp
from lifelines import KaplanMeierFitter, CoxPHFitter
from math import pi
#sklearn
import sklearn
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.utils import Bunch
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.calibration import IsotonicRegression, calibration_curve
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance

#modelos
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


#metrics
from sklearn.metrics import average_precision_score, roc_auc_score, brier_score_loss,confusion_matrix,classification_report
from fairlearn.metrics import MetricFrame


#optimization momdels
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer

warnings.filterwarnings('ignore')
RNG_SEED = 42
np.random.seed(RNG_SEED)


# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

logger.info("✓ Librerías importadas y configuración completa")

# Data Load

In [19]:
client = bigquery.Client(project="spin-aip-singularity-comp-sb")

query = """ SELECT * FROM `spin-aip-singularity-comp-sb.model_activation.dataste_model_activation_timewindow_30D_V-1-5-0`"""

data = client.query(query).to_dataframe()



In [20]:
data.head()

,user_id,signup_date,signup_ts,userTypeIdentifier,channelUserIdentifier,accountLevel,stateName,gender,user_type,channelDetail,birth_date,birthState,Card_linked_date,IsActive,phn_confir,email_confir,phone_conf_ts,email_conf_ts,phn_confir_d7,email_confir_d7,both_confir_d7,premia_accountid,has_premia,activation_date_ever,activation_date_30d,label_activated_30d,tx_30d_count,tx_30d_amount,label_5tx_30d,first_tx_type,first_tx_amount,activation_channel,latest_tx_date,lifespan_days,days_since_last,tx_30d_from_activation,days_to_first_activation
0,00010f87-83f3-427d-b76e-e7b4cda95d7c,2025-06-04,2025-06-04 22:20:00.717000+00:00,3,1,3,EM,female,HYBRID,POS,1984-06-22,DF,2025-06-04,True,1,1,2025-11-09 00:54:41.078000+00:00,2025-06-05 04:32:57.298000+00:00,0,1,0,M3OC2BJ,1,2025-06-04,2025-06-04,1,55,107541.3400,1,TRANSFER_TO_CLABE,13800.0000,SPEI/Transfer,2025-11-20,170,1,55,0
1,0001f91d-3db1-442a-a5ab-a7b7849e2ae9,2025-05-13,2025-05-13 19:54:54.464000+00:00,3,1,2,EM,female,HYBRID,POS,1994-04-02,OC,2025-05-13,True,1,0,2025-05-14 09:17:35.776000+00:00,NaT,1,0,0,UPBWNOT,1,2025-05-13,2025-05-13,1,32,25135.3600,1,CASH_IN_AT_OXXO,1985.0000,CashIn_OXXO,2025-11-20,192,1,32,0
2,000399d4-4587-4726-a257-7cd565c4e6af,2025-01-29,2025-01-29 20:40:45.420000+00:00,3,2,3,GT,male,HYBRID,ORGANIC,1988-07-22,GT,2025-03-20,True,1,1,2025-01-30 02:40:51.522000+00:00,2025-01-30 04:08:32.691000+00:00,1,1,1,LO82LV8,1,2025-03-01,NaT,0,<NA>,NaN,0,SPEI_CASH_IN,1950.0000,SPEI/Transfer,2025-11-16,261,5,33,31
3,0003ae93-ceb6-41e4-a319-beb89d0c5dbb,2025-05-04,2025-05-05 02:53:01.035000+00:00,3,1,2,JA,female,HYBRID,POS,2001-05-31,JC,2025-05-04,True,1,0,2025-05-05 23:29:16.216000+00:00,NaT,1,0,0,Z282OQG,1,2025-05-05,2025-05-05,1,35,41568.0800,1,SPEI_CASH_IN,10430.0000,SPEI/Transfer,2025-08-03,91,110,35,1
4,00050c54-e6ea-463e-8b8b-280e90b6b35f,2025-04-18,2025-04-18 16:22:31.820000+00:00,3,1,2,EM,male,HYBRID,POS,2006-03-31,MC,2025-04-18,True,1,0,2025-04-19 01:19:45.717000+00:00,NaT,1,0,0,QVXA7KV,1,2025-04-18,2025-04-18,1,31,18175.2500,1,CASH_IN_AT_OXXO,2000.0000,CashIn_OXXO,2025-11-20,217,1,31,0


In [21]:
df = data.copy(deep=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2384265 entries, 0 to 2384264
Data columns (total 37 columns):
 #   Column                    Dtype              
---  ------                    -----              
 0   user_id                   object             
 1   signup_date               dbdate             
 2   signup_ts                 datetime64[us, UTC]
 3   userTypeIdentifier        Int64              
 4   channelUserIdentifier     Int64              
 5   accountLevel              Int64              
 6   stateName                 object             
 7   gender                    object             
 8   user_type                 object             
 9   channelDetail             object             
 10  birth_date                dbdate             
 11  birthState                object             
 12  Card_linked_date          dbdate             
 13  IsActive                  boolean            
 14  phn_confir                Int64              
 15  email_confir   

In [22]:
df.shape

(2384265, 37)

# Feature Engineering

In [23]:
@dataclass
class Config:
    project_id: str = "spin-aip-singularity-comp-sb"
    table_fqn: str = "spin-aip-singularity-comp-sb.model_activation.dataste_model_activation_timewindow_30D_V-1-5-0"
    label_col: str = "label_5tx_30d"
    signup_ts_col: str = "signup_ts"
    signup_date_col: str = "signup_date"
    tz_local: str = "America/Mexico_City"
    embargo_days: int = 3      
    holdout_days: int = 14 
    n_splits: int = 5
    train_sample_frac: float = 1.0
    activation_filter: int = 1
    random_state: int = RNG_SEED
    lift_fracs: t.Tuple[float, ...] = (0.01, 0.02, 0.05, 0.10)

CFG = Config()

In [24]:
# Leakage / IDs a ignorar en features
LEAKY_ALWAYS = {    
    #"label_activated_30d"
    "y_w0",
    "y_w1",
    "y_w7",
    "y_w30",
    "y_cum30",
    "label_5tx_30d",
    # info post-activación / post-window
    #"activation_date_ever",
    #"activation_date_30d",
    #"days_to_first_activation",
    #"tx_30d_count",
    #"tx_30d_amount",
    #"tx_30d_from_activation", 
    #"first_tx_type",
    #"first_tx_amount",
    #"latest_tx_date",
    # confirmaciones si no son estrictamente previas al cutoff de cada horizonte
#    "phn_confir","email_confir",
    #"phn_confir_d7",
    #"email_confir_d7",
    #"both_confir_d7",
    #'Card_linked_date',
    #"activation_*",
   # "*_30d_*",
    "*latest_tx*",
    #ID
    "user_id", 
    "userid",
    "channelUserIdentifier",
    "premia_accountid", 
    "accountid", 
    "member_id", 
    "spin_user_id", 
    "id"
}

In [25]:
LEAK_BAN = {
    #"activation_date_30d"
    "y_w0",
    "y_w1",
    "y_w7",
    "y_w30",
    "y_cum30",
    "label_activated_30d",
    "label_5tx_30d",
    #"activation_date_ever",
    #"days_to_first_activation",
    #"latest_tx_date",
    #"tx_30d_count",
    #"tx_30d_amount",
    #"tx_30d_from_activation",
    #"first_tx_type",
    #"first_tx_amount",
    #"activation_channel"
}

In [26]:
# Regex anti-leak (además del set LEAKY_ALWAYS existente)
LEAK_PATTERNS = [
    #r"(^|_)activation(_|$)", 
    #r"(^|_)first_tx(_|$)", 
    #r"(^|_)latest_tx(_|$)",
    #r"(^|_)tx_30d(_|$)", 
    #r_)days_to_first(_|$)", 
 #   r"(^|_)from_activation(_|$)"
]

In [27]:
def assert_no_regex_leak(df_like: pd.DataFrame):
    bad = []
    for c in df_like.columns:
        for pat in LEAK_PATTERNS:
            if re.search(pat, c, flags=re.IGNORECASE):
                bad.append(c); break
    assert len(bad) == 0, f"LEAKAGE by regex: quita columnas {sorted(set(bad))}"

def assert_no_labelish_cols(df_like):
    inter = [c for c in df_like.columns if c in LEAK_BAN]
    assert len(inter) == 0, f"LEAKAGE: quita columnas {inter}"


EXTRA_DROP_TS = {"phone_conf_ts", "email_conf_ts"}

# Normalización de estados (stateName -> siglas)
def _norm(s: str) -> str:
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    s = re.sub(r"[^A-Z ]", "", s.upper())
    return re.sub(r"\s+", " ", s).strip()

STATE_TO_ABBR = {
    "AGUASCALIENTES":"AG","BAJA CALIFORNIA":"BC","BAJA CALIFORNIA SUR":"BS","CAMPECHE":"CM",
    "CHIAPAS":"CS","CHIHUAHUA":"CH","CIUDAD DE MEXICO":"DF","COAHUILA":"CO","COLIMA":"CL",
    "DURANGO":"DG","GUANAJUATO":"GT","GUERRERO":"GR","HIDALGO":"HG","JALISCO":"JA","MEXICO":"EM",
    "MICHOACAN":"MI","MORELOS":"MO","NAYARIT":"NA","NUEVO LEON":"NL","OAXACA":"OA","PUEBLA":"PU",
    "QUERETARO":"QT","QUINTANA ROO":"QR","SAN LUIS POTOSI":"SL","SINALOA":"SI","SONORA":"SO",
    "TABASCO":"TB","TAMAULIPAS":"TM","TLAXCALA":"TL","VERACRUZ":"VE","YUCATAN":"YU","ZACATECAS":"ZA"
}
STATE_SYNONYMS = {"CDMX":"CIUDAD DE MEXICO","ESTADO DE MEXICO":"MEXICO","EDOMEX":"MEXICO"}

# birthState canon + buckets regionales
CANON = {
    "SR":"SO","SO":"SO","VZ":"VE","VE":"VE","YN":"YU","YU":"YU","JC":"JA","JA":"JA","MC":"MI","MI":"MI",
    "TS":"TM","TM":"TM","TC":"TB","TB":"TB","CC":"CL","CL":"CL","DF":"DF","EM":"EM","NL":"NL","BC":"BC",
    "BS":"BS","SI":"SI","NA":"NA","DG":"DG","ZA":"ZA","AG":"AG","SL":"SL","HG":"HG","MO":"MO","TL":"TL",
    "PU":"PU","QT":"QT","GT":"GT","OA":"OA","CM":"CM","CS":"CS","CO":"CO","GR":"GR","QR":"QR","CH":"CH",
    "MS":"MI","MN":"MI","SP":"SL","NE":"NL","OC":"OA","PL":"PU","NT":"NA","ZS":"ZA","AS":"AG","UN":"OT", None:"OT"
}
REGION_BUCKET = {
    "BC":1,"SO":1,"CH":1,"CO":1,"NL":1,"TM":1,                # Norte
    "BS":2,"SI":2,"NA":2,"DG":2,"ZA":2,                       # Norte-Occidente
    "JA":3,"AG":3,"CL":3,"MI":3,"SL":3,                       # Centro-Norte
    "DF":4,"EM":4,"HG":4,"MO":4,"TL":4,"PU":4,"QT":4,"GT":4,  # Centro-País
    "CS":5,"TB":5,"CM":5,"YU":5,"QR":5,"OA":5,"GR":5,"VE":5,  # Sur-Sureste
    "OT":0
}

# Mapeos categóricos
GENDER_MAP = {"female":1, "male":0}
USER_TYPE_MAP = {"HYBRID":0, "DIGITAL":1, "ANALOG":2}
CHANNEL_DETAIL_MAP = {"ORGANIC":0,"COLLABORATOR":1,"POS":2,"SPIN_PREMIA":3,"DIGITAL_ORGANIC":4,"DIGITAL":5}


CASH_IN_TX = {
        "CASH_IN_AT_OXXO",
        "CASH_IN_AT_OXXO_QR",
        "CASH_OUT_WITH_CARD_AT_OXXO", 
        "CASH_OUT_AT_OXXO",
        "CASH_OUT_AT_MERCHANT",
        "CARD_PURCHASE",
        "CARD_ATM_WITHDRAWAL"
}
SPEI_TX = {
        "SPEI_CASH_IN",
        "TRANSFER_TO_CARD",
        "TRANSFER_TO_CLABE",
        "P2P_TRANSFER_TARGET",
        "P2P_TRANSFER_TARGET_CLABE",
        "P2P_TRANSFER_TARGET",
        "P2P_TRANSFER_TARGET_CARD"}

P2P_TX ={
        "P2P_TRANSFER_TARGET_CLABE",
         "P2P_TRANSFER_TARGET",
         "P2P_TRANSFER_TARGET_CARD",
         "P2P_TRANSFER_SOURCE_CARD",
         "P2P_TRANSFER_SOURCE_CLABE",
         "P2P_TRANSFER_SOURCE",
         "IN_APP_PURCHASE_TAE",
         "IN_APP_PURCHASE_BILLPAYMENT",
         "QR_MERCHANT_PAYMENT",
         "GIFT_CARD_PURCHASE",
         "INTERNATIONAL_REMITTANCE_CASH_IN"}

TX_TYPE_MAP = {
    **{k: 0 for k in CASH_IN_TX},  # 0 = Cash_In
    **{k: 1 for k in SPEI_TX},     # 1 = SPEI
    **{k: 2 for k in P2P_TX}       # 2 = P2P
}

In [28]:
class FeatureBuilder:
    def __init__(self, cfg: Config):
        self.cfg = cfg
        self.ohe_states_: t.List[str] = []
        # Pipeline de normalización
        self.scaler_: RobustScaler | None = None
        self.numeric_cols_: t.List[str] = []
        self._fitted_scaler: bool = False

    @staticmethod
    def _state_to_abbr(s: t.Any) -> str:
        if pd.isna(s): return "OT"
        s = str(s)
        if s.upper() in REGION_BUCKET: return s.upper()
        s2 = _norm(s)
        s2 = STATE_SYNONYMS.get(s2, s2)
        return STATE_TO_ABBR.get(s2, "OT")

    @staticmethod
    def _canon_birthstate(s: t.Any) -> str:
        if pd.isna(s): return "OT"
        s = str(s).upper()
        return CANON.get(s, s if s in REGION_BUCKET else "OT")

    def _mk_time_feats(self, df: pd.DataFrame) -> pd.DataFrame:
        ts = pd.to_datetime(df[self.cfg.signup_ts_col], utc=True).dt.tz_convert(ZoneInfo(self.cfg.tz_local))
        df = df.copy()
        df["signup_dow"] = ts.dt.weekday.astype("int16")
        df["signup_week"] = ts.dt.isocalendar().week.astype("int16")
        df["signup_month"] = ts.dt.month.astype("int16")
        hr = ts.dt.hour
        df["signup_daypart"] = np.select([(hr>=5)&(hr<=11),(hr>=12)&(hr<=17)],[0,1],default=2).astype("int8")
        years = list({d.year for d in pd.to_datetime(df[self.cfg.signup_date_col]).dt.date})
        mx_hol = holidays.MX(years=years)
        dates = pd.to_datetime(df[self.cfg.signup_date_col]).dt.date
        df["is_holiday_mx"] = dates.map(lambda d: 1 if d in mx_hol else 0).astype("int8")
        day = ts.dt.day
        eom = (ts + pd.offsets.MonthEnd(0)).dt.day
        df["near_payday_any"] = ((np.abs(day-1)<=3)|(np.abs(day-15)<=3)|(np.abs(day-eom)<=3)).astype("int8")
        df["near_payday_1st"] = (np.abs(day-1)<=3).astype("int8")
        df["near_payday_15"]  = (np.abs(day-15)<=3).astype("int8")
        df["near_payday_eom"] = (np.abs(day-eom)<=3).astype("int8")
        return df

    def fit(self, df: pd.DataFrame):
        st = df["stateName"].map(self._state_to_abbr)
        self.ohe_states_ = sorted(st.dropna().unique().tolist())
        if "OT" not in self.ohe_states_: self.ohe_states_.append("OT")
        return self

    def transform(self, df: pd.DataFrame) -> Bunch:
        df = df.copy()
        s_ts = pd.to_datetime(df[CFG.signup_ts_col], utc=True)

        # Categóricas core
        df["gender_bin"] = df["gender"].map(GENDER_MAP).astype("float32")
        df["user_type_tri"] = df["user_type"].map(USER_TYPE_MAP).astype("float32")
        df["channel_detail_code"] = df["channelDetail"].map(CHANNEL_DETAIL_MAP).astype("float32")

        # birth bucket + edad
        bcanon = df["birthState"].map(self._canon_birthstate)
        df["birth_bucket"] = bcanon.map(REGION_BUCKET).astype("float32")
        bdate = pd.to_datetime(df["birth_date"], errors="coerce", utc=True)
        df["age_years"] = ((s_ts - bdate).dt.days/365.25).astype("float32")

        # Time features
        df = self._mk_time_feats(df)

        # state OHE
        st = df["stateName"].map(self._state_to_abbr)
        for ab in self.ohe_states_:
            df[f"state_{ab}"] = (st==ab).astype("int8")

        # Confirmaciones / flags: Int nulos a 0
        if "phone_conf_ts" in df.columns:
            phn_ts = pd.to_datetime(df["phone_conf_ts"], errors="coerce", utc=True)
            df["phn_confir"] = (phn_ts < s_ts).fillna(False).astype("int8")
        else:
            df["phn_confir"] = 0

        if "email_conf_ts" in df.columns:
            email_ts = pd.to_datetime(df["email_conf_ts"], errors="coerce", utc=True)
            df["email_confir"] = (email_ts < s_ts).fillna(False).astype("int8")
        else:
            df["email_confir"] = 0

        # Card_linked_date -> deltas sin fuga
        if "Card_linked_date" in df.columns:
            card_dt = pd.to_datetime(df["Card_linked_date"], errors="coerce", utc=True)
            # Cambiar <= por < (estrictamente antes)
            before = card_dt < s_ts  
            lag_days = (s_ts - card_dt).dt.days.astype("float32")
            df["card_linked_before_signup"] = before.fillna(False).astype("int8")
            # Solo crear lag_days si ocurrió ANTES (no <=)
            df["card_linked_lag_days"] = np.where(before, lag_days, np.nan).astype("float32")
            df = df.drop(columns=["Card_linked_date"])

        LEAKY_FEATURES = {
            #'lifespan_days',        
            #'days_since_last',
            'tx_30d_count',
            'tx_30d_from_activation',
            #'p_activation_30d', 
            'p_tx_0', 
            'p_tx_1', 
            'p_tx_2'
        }
            
        # Armar X
        drop_cols = set(LEAKY_ALWAYS) | {
            "stateName","gender","user_type","channelDetail","birthState","birth_date",
            CFG.signup_date_col, CFG.signup_ts_col
        } | set(EXTRA_DROP_TS) | LEAKY_FEATURES  

        drop_cols = [c for c in drop_cols if c in df.columns]
        X = df.drop(columns=drop_cols, errors="ignore")

        # Limpiar tipos
        # 1) Si queda algún dtype extension de BigQuery -> fuera
        bad_ext = [c for c in X.columns if "db_dtypes" in str(X[c].dtype).lower() or "dbdate" in str(X[c].dtype).lower()]
        X = X.drop(columns=bad_ext, errors="ignore")

        # 2) Casts seguros
        for c in X.columns:
            if pd.api.types.is_integer_dtype(X[c]) or str(X[c].dtype).startswith("Int"):
                X[c] = pd.to_numeric(X[c], errors="coerce").fillna(0).astype("float32")
            elif pd.api.types.is_float_dtype(X[c]):
                X[c] = X[c].astype("float32")
            elif pd.api.types.is_bool_dtype(X[c]):
                X[c] = X[c].astype("int8")
            elif pd.api.types.is_datetime64_any_dtype(X[c]):
                X = X.drop(columns=[c])

        # 3) Objetos -> fuera
        obj_cols = X.select_dtypes(include=["object"]).columns.tolist()
        if obj_cols:
            X = X.drop(columns=obj_cols)

        # Normalización con RobustScaler
        if not self._fitted_scaler:
            # Primera vez: identificar columnas numéricas y fit scaler
            numeric_cols = [c for c in X.columns
                            if not c.startswith('state_')  # Excluir one-hot de estados
                            and X[c].dtype == 'float32'     # Solo float32
                            and X[c].nunique() > 10]        # Excluir binarias
            
            self.numeric_cols_ = numeric_cols
            
            if len(numeric_cols) > 0:
                self.scaler_ = RobustScaler()
                self.scaler_.fit(X[numeric_cols])
                self._fitted_scaler = True
                print(f'RobustScaler fitted con {len(numeric_cols)} features numéricas')
            else:
                print('No se encontraron features numéricas para normalizar')
                self._fitted_scaler = True
        
        # Aplicar normalización si existe
        if self.scaler_ is not None and len(self.numeric_cols_) > 0:
            existing_cols = [c for c in self.numeric_cols_ if c in X.columns]
            if len(existing_cols) > 0:
                X[existing_cols] = self.scaler_.transform(X[existing_cols])

        # y binaria
        y = pd.to_numeric(df[CFG.label_col], errors="coerce").fillna(0).astype(int).values
        
        # Verificación final de leakage
        assert_no_regex_leak(X)
        LEAK_BAN = LEAKY_ALWAYS  | {"label_5tx_30d"}
        #assert_no_labelish_cols(X)
        
        meta = pd.DataFrame({
            "user_id": df.get("user_id", pd.Series(index=df.index, dtype="object")),
            "signup_date": pd.to_datetime(df[CFG.signup_date_col], errors="coerce"),
            "gender": df["gender"].astype(str),
            "channelDetail": df["channelDetail"].astype(str),
            "state_abbr": st.astype(str)
        })
        return Bunch(X=X, y=y, meta=meta)

In [35]:
df_5tx = df[df["label_activated_30d"]==1].reset_index(drop=True)


print(f"Threshold de Activación aplicado: {CFG.activation_filter}")
print("Shape df_5tx (activados y high-score):", df_5tx.shape)
print("Rate 5tx en esta población:", df_5tx["label_5tx_30d"].mean())

df_5tx
# Construcción de features con FeatureBuilder
fb = FeatureBuilder(CFG).fit(df_5tx)
bunch = fb.transform(df_5tx)

X_all = bunch.X.values.astype("float32") 
feature_names = list(bunch.X.columns)  # Guardar nombres para después

#X_all = bunch.X.astype("float32")       
y_all = df_5tx["label_5tx_30d"].astype(int).values  

signup_dates = pd.to_datetime(df_5tx[CFG.signup_date_col], errors="coerce")

print("X_all shape:", X_all.shape)
print("y_all rate (5tx):", y_all.mean())

Threshold de Activación aplicado: 1
Shape df_5tx (activados y high-score): (1470563, 37)
Rate 5tx en esta población: 0.6919791943629753
RobustScaler fitted con 8 features numéricas
X_all shape: (1470563, 64)
y_all rate (5tx): 0.6919791943629753


In [36]:
feature_names

['userTypeIdentifier',
 'accountLevel',
 'IsActive',
 'phn_confir',
 'email_confir',
 'phn_confir_d7',
 'email_confir_d7',
 'both_confir_d7',
 'has_premia',
 'label_activated_30d',
 'tx_30d_amount',
 'first_tx_amount',
 'lifespan_days',
 'days_since_last',
 'days_to_first_activation',
 'gender_bin',
 'user_type_tri',
 'channel_detail_code',
 'birth_bucket',
 'age_years',
 'signup_dow',
 'signup_week',
 'signup_month',
 'signup_daypart',
 'is_holiday_mx',
 'near_payday_any',
 'near_payday_1st',
 'near_payday_15',
 'near_payday_eom',
 'state_AG',
 'state_BC',
 'state_BS',
 'state_CH',
 'state_CL',
 'state_CM',
 'state_CO',
 'state_CS',
 'state_DF',
 'state_DG',
 'state_EM',
 'state_GR',
 'state_GT',
 'state_HG',
 'state_JA',
 'state_MI',
 'state_MO',
 'state_NA',
 'state_NL',
 'state_OA',
 'state_OT',
 'state_PU',
 'state_QR',
 'state_QT',
 'state_SI',
 'state_SL',
 'state_SO',
 'state_TB',
 'state_TL',
 'state_TM',
 'state_VE',
 'state_YU',
 'state_ZA',
 'card_linked_before_signup',
 'c

In [37]:
# MATRIZ DE CORRELACIÓN PEARSON + SPEARMAN CONTRA EL TARGET

# Convertimos X a DataFrame con nombres de features
if hasattr(bunch.X, "columns"):
    X_df = bunch.X.copy()
    cols = list(X_df.columns)
else:
    cols = getattr(bunch, "feature_names", [f"f_{i}" for i in range(bunch.X.shape[1])])
    X_df = pd.DataFrame(bunch.X, columns=cols)

# Target
y = df_5tx["label_5tx_30d"].astype(int)

# PEARSON
pearson_corr = X_df.apply(lambda col: col.corr(y, method="pearson"))

# SPEARMAN
spearman_corr = X_df.apply(lambda col: col.corr(y, method="spearman"))

# Compactamos en un solo DataFrame
corr_df = pd.DataFrame({
    "feature": cols,
    "pearson": pearson_corr.values,
    "spearman": spearman_corr.values
})

# Ordenar por la correlación absoluta más alta
corr_df["abs_pearson"] = corr_df["pearson"].abs()
corr_df["abs_spearman"] = corr_df["spearman"].abs()

corr_df_sorted = corr_df.sort_values(by=["abs_pearson", "abs_spearman"], ascending=False)

print("Top 20 features más correlacionadas con label_5tx_30d:")
display(corr_df_sorted.head(20))

# FLAGS DE ALERTA
THRESHOLD_ALERT = 0.60  # ajusta según el nivel de paranoia

suspicious = corr_df_sorted[
    (corr_df_sorted["abs_pearson"] > THRESHOLD_ALERT) |
    (corr_df_sorted["abs_spearman"] > THRESHOLD_ALERT)
]

if suspicious.shape[0] > 0:
    print("\n⚠️ ALERTA: Features con correlación ALTA con el target")
    display(suspicious)
else:
    print("\n✔️ No se detectaron features con correlación fuerte (Pearson o Spearman) contra el target.")


Top 20 features más correlacionadas con label_5tx_30d:


,feature,pearson,spearman,abs_pearson,abs_spearman
13,days_since_last,-0.2990,-0.3378,0.2990,0.3378
10,tx_30d_amount,0.2454,0.5535,0.2454,0.5535
12,lifespan_days,0.2388,0.2824,0.2388,0.2824
14,days_to_first_activation,-0.2371,-0.1330,0.2371,0.1330
0,userTypeIdentifier,0.2284,0.2206,0.2284,0.2206
16,user_type_tri,-0.2284,-0.2206,0.2284,0.2206
5,phn_confir_d7,0.1806,0.1806,0.1806,0.1806
19,age_years,-0.1490,-0.1385,0.1490,0.1385
1,accountLevel,0.1274,0.1288,0.1274,0.1288
7,both_confir_d7,0.1174,0.1174,0.1174,0.1174



✔️ No se detectaron features con correlación fuerte (Pearson o Spearman) contra el target.


# Models

In [38]:
# Orden cronológico y definición de train / embargo / holdout 
order = np.argsort(signup_dates.values)
signup_dates = signup_dates.iloc[order].reset_index(drop=True)
X_all = X_all[order]
y_all = y_all[order]

max_date = signup_dates.max()
holdout_start = max_date - pd.Timedelta(days=CFG.holdout_days)
embargo_end = holdout_start
train_end = holdout_start - pd.Timedelta(days=CFG.embargo_days)

train_mask = signup_dates <= train_end
holdout_mask = signup_dates > holdout_start
embargo_mask = (~train_mask) & (~holdout_mask)

print("Última fecha:", max_date.date())
print("Train hasta:", train_end.date())
print("Embargo entre:", train_end.date(), "y", holdout_start.date())
print("Holdout desde:", holdout_start.date())
print("N train   :", train_mask.sum())
print("N embargo :", embargo_mask.sum())
print("N holdout :", holdout_mask.sum())


Última fecha: 2025-10-21
Train hasta: 2025-10-04
Embargo entre: 2025-10-04 y 2025-10-07
Holdout desde: 2025-10-07
N train   : 1378919
N embargo : 16420
N holdout : 75224


In [39]:
X_train_full = X_all[train_mask.values]
y_train_full = y_all[train_mask.values]

n_train = X_train_full.shape[0]
split_idx = int(n_train * 0.8)

X_tr, X_val = X_train_full[:split_idx], X_train_full[split_idx:]
y_tr, y_val = y_train_full[:split_idx], y_train_full[split_idx:]

print("Train interno:", X_tr.shape[0], "obs")
print("Valid interno:", X_val.shape[0], "obs")


Train interno: 1103135 obs
Valid interno: 275784 obs


In [40]:
# Modelo
model_5tx =  LGBMClassifier(
    n_estimators=400,
    max_depth=8,
    learning_rate=0.05,
    class_weight="balanced",
    n_jobs=-1,
    random_state=CFG.random_state,
    verbose=-1
)

model_5tx.fit(X_tr, y_tr)

p_val = model_5tx.predict_proba(X_val)[:, 1]

print("AP (valid):", average_precision_score(y_val, p_val))
print("AUC (valid):", roc_auc_score(y_val, p_val))
print("Brier (valid):", brier_score_loss(y_val, p_val))

AP (valid): 0.9831110028924769
AUC (valid): 0.9610318124660406
Brier (valid): 0.07727442233754217


In [41]:
def objective_temporal_single(trial, model_name, X_tr, y_tr, X_val, y_val):
    """Objective con un solo split temporal train/val."""
    model = build_model(model_name, trial)

    model.fit(X_tr, y_tr)
    p_val = model.predict_proba(X_val)[:, 1]

    ap = average_precision_score(y_val, p_val)
    return ap

In [42]:
def lift_at_fracs(y_true: np.ndarray, y_score: np.ndarray, fracs) -> dict:
    """Devuelve capture_rate y lift para cada fracción."""
    order = np.argsort(-y_score)
    y_sorted = y_true[order]
    total_pos = y_true.sum()

    res = {}
    for f in fracs:
        k = max(1, int(len(y_true) * f))
        captured = y_sorted[:k].sum()
        if total_pos > 0:
            capture_rate = captured / total_pos
            lift = capture_rate / f
        else:
            capture_rate, lift = np.nan, np.nan
        res[f] = {
            "capture_rate": float(capture_rate),
            "lift": float(lift),
            "n_users": int(k),
        }
    return res

In [43]:
#  Evaluación en HOLDOUT 
X_holdout = X_all[holdout_mask.values]
y_holdout = y_all[holdout_mask.values]

p_holdout = model_5tx.predict_proba(X_holdout)[:, 1]

metrics_val = {
    "AP": float(average_precision_score(y_val, p_val)),
    "AUC": float(roc_auc_score(y_val, p_val)),
    "Brier": float(brier_score_loss(y_val, p_val)),
    "Lift": lift_at_fracs(y_val, p_val, CFG.lift_fracs),
}

metrics_holdout = {
    "AP": float(average_precision_score(y_holdout, p_holdout)),
    "AUC": float(roc_auc_score(y_holdout, p_holdout)),
    "Brier": float(brier_score_loss(y_holdout, p_holdout)),
    "Lift": lift_at_fracs(y_holdout, p_holdout, CFG.lift_fracs),
}

print("\n MÉTRICAS 5TX (VALID) ")
print(json.dumps(metrics_val, indent=2))
print("\n MÉTRICAS 5TX (HOLDOUT) ")
print(json.dumps(metrics_holdout, indent=2))


 MÉTRICAS 5TX (VALID) 
{
  "AP": 0.9831110028924769,
  "AUC": 0.9610318124660406,
  "Brier": 0.07727442233754217,
  "Lift": {
    "0.01": {
      "capture_rate": 0.01409891228195779,
      "lift": 1.409891228195779,
      "n_users": 2757
    },
    "0.02": {
      "capture_rate": 0.028202938424010594,
      "lift": 1.4101469212005298,
      "n_users": 5515
    },
    "0.05": {
      "capture_rate": 0.070509902990074,
      "lift": 1.4101980598014798,
      "n_users": 13789
    },
    "0.1": {
      "capture_rate": 0.1409891228195779,
      "lift": 1.409891228195779,
      "n_users": 27578
    }
  }
}

 MÉTRICAS 5TX (HOLDOUT) 
{
  "AP": 0.9843341939783573,
  "AUC": 0.9631101294601718,
  "Brier": 0.07652707499366809,
  "Lift": {
    "0.01": {
      "capture_rate": 0.014037969721294032,
      "lift": 1.4037969721294032,
      "n_users": 752
    },
    "0.02": {
      "capture_rate": 0.028075939442588065,
      "lift": 1.4037969721294032,
      "n_users": 1504
    },
    "0.05": {
      "

In [44]:
# Esto genera el score para las 1.4M filas, respetando el orden cronológico ya establecido

# Predecir la probabilidad condicional para TODOS los usuarios activados (X_all)


p_cond_all = model_5tx.predict_proba(X_all)[:, 1]

# 2. Crear el DataFrame de scores copiando el original ordenado
df_5tx_scores = df_5tx.iloc[order].copy()

# 3. Asignar la probabilidad condicional (Modelo 2)
df_5tx_scores["p_5tx_30d"] = p_cond_all


# Guardar resultados (Artefacto de Producción)
# Seleccionamos solo IDs y scores para no hacer el archivo gigante
cols_export = ["user_id", "signup_date", "p_5tx_30d"]
df_export = df_5tx_scores[cols_export]


In [45]:
df_export

,user_id,signup_date,p_5tx_30d
316337,26a4836f-0602-47ee-8192-5b845af1ec00,2025-01-01,0.0074
297552,cd139aff-6dd2-4480-9722-81ef91b00ac5,2025-01-01,0.1183
895779,08f6b097-4731-4603-80fa-cb19498eaf60,2025-01-01,0.0954
670465,8373d5d3-4822-4dd7-b756-92312a44e524,2025-01-01,0.9224
1324641,66f79004-b007-4839-8fb7-817274995bb7,2025-01-01,0.9412
...,...,...,...
1269067,1fb4bf96-74b4-47a1-a2c0-ac6d7cb68ccf,2025-10-21,0.8211
546996,13bddb98-7501-4101-a404-a1b713ee21c3,2025-10-21,0.1382
778872,a2bcb966-e2d6-4f05-a947-8a5d58a0f31d,2025-10-21,0.8606
686981,92e91172-75a9-416f-9060-531029f733ed,2025-10-21,0.4351


# Artifacts

In [46]:
from pathlib import Path
import json, sys, platform, cloudpickle
from datetime import datetime, timezone

In [47]:
ARTIFACTS_DIR = Path("mlops_artifacts_v2")
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

In [48]:
MODEL_PATH = ARTIFACTS_DIR / "model_5tx.pkl"
FB_PATH = ARTIFACTS_DIR / "feature_builder.pkl"
CFG_PATH = ARTIFACTS_DIR / "cfg.pkl"
SCHEMA_PATH = ARTIFACTS_DIR / "feature_schema.json"

In [50]:

#guardar modelo 
with open(MODEL_PATH, "wb") as f:
    cloudpickle.dump(model_5tx, f)  
print(f" Modelo guardado: {MODEL_PATH} ({MODEL_PATH.stat().st_size:,} bytes)")

# 2. GUARDAR FeatureBuilder
with open(FB_PATH, "wb") as f:
    cloudpickle.dump(fb, f)
print(f"FeatureBuilder guardado: {FB_PATH} ({FB_PATH.stat().st_size:,} bytes)")

# 3. GUARDAR Config
with open(CFG_PATH, "wb") as f:
    cloudpickle.dump(CFG, f)
print(f" Config guardada: {CFG_PATH}")

# 4. GUARDAR Schema

feature_schema = list(bunch.X.columns)  
SCHEMA_PATH.write_text(json.dumps(feature_schema, indent=2), encoding="utf-8")
print(f"Schema guardado: {len(feature_schema)} features")
print(f" {feature_schema[:5]}")

 Modelo guardado: mlops_artifacts_v2/model_5tx.pkl (1,445,064 bytes)
FeatureBuilder guardado: mlops_artifacts_v2/feature_builder.pkl (15,036 bytes)
 Config guardada: mlops_artifacts_v2/cfg.pkl
Schema guardado: 64 features
 ['userTypeIdentifier', 'accountLevel', 'IsActive', 'phn_confir', 'email_confir']


In [51]:
import os
from pathlib import Path

ARTIFACTS_DIR = Path("mlops_artifacts_v2")

for f in ARTIFACTS_DIR.glob("*"):
    size = os.path.getsize(f)
    print(f"{f.name}: {size:,} bytes")

.ipynb_checkpoints: 4,096 bytes
cfg.pkl: 4,768 bytes
feature_schema.json: 1,088 bytes
feature_builder.pkl: 15,036 bytes
model_5tx.pkl: 1,445,064 bytes


# MLOps inference

In [52]:
import json, sys, platform, cloudpickle  
from pathlib import Path
from datetime import datetime, timezone

In [53]:
ARTIFACTS_DIR = Path("mlops_artifacts_v2")
MODEL_PATH = ARTIFACTS_DIR / "model_5tx.pkl"
FB_PATH = ARTIFACTS_DIR / "feature_builder.pkl"
CFG_PATH = ARTIFACTS_DIR / "cfg.pkl"
SCHEMA_PATH = ARTIFACTS_DIR / "feature_schema.json"

In [54]:
# Cargar Feature Builder
with open(FB_PATH, "rb") as f:
    fb_loaded = cloudpickle.load(f)
print(f"FeatureBuilder: {type(fb_loaded).__name__}")

# Cargar Modelo
with open(MODEL_PATH, "rb") as f:
    model_loaded = cloudpickle.load(f)
print(f"Modelo: {type(model_loaded).__name__}")

# Cargar Config
cfg_loaded = None
if CFG_PATH.exists():
    with open(CFG_PATH, "rb") as f:
        cfg_loaded = cloudpickle.load(f)
    print(f"Config: {type(cfg_loaded).__name__}")
else:
    print("Config no encontrada (opcional)")

# Cargar Schema
schema_loaded = json.loads(SCHEMA_PATH.read_text(encoding="utf-8"))
print(f"Schema: {len(schema_loaded)} features")

# Validación de schema no debe estar vacio
if len(schema_loaded) == 0:
    raise ValueError("CRÍTICO: feature_schema.json está vacío!")


FeatureBuilder: FeatureBuilder
Modelo: LGBMClassifier
Config: Config
Schema: 64 features


In [55]:
# Extraer features del modelo
if hasattr(model_loaded, "feature_name_") and model_loaded.feature_name_:
    model_features = list(model_loaded.feature_name_)
elif hasattr(model_loaded, "booster_"):
    model_features = list(model_loaded.booster_.feature_name())
else:
    model_features = []
    print("No se pudieron extraer features del modelo")

# Comparar
if model_features:
    schema_set = set(schema_loaded)
    model_set = set(model_features)
    
    missing_in_schema = model_set - schema_set
    extra_in_schema = schema_set - model_set
    
    if missing_in_schema:
        print(f"Features en modelo pero no en schema: {len(missing_in_schema)}")
    if extra_in_schema:
        print(f"Features en schema pero no en modelo: {len(extra_in_schema)}")
    
    if not missing_in_schema and not extra_in_schema:
        print(f"Schema consistente: {len(model_features)} features")


Features en modelo pero no en schema: 64
Features en schema pero no en modelo: 64


In [56]:
from google.cloud import bigquery
project_id = cfg_loaded.project_id if cfg_loaded else "spin-aip-singularity-comp-sb"
table_fqn = cfg_loaded.table_fqn if cfg_loaded else "spin-aip-singularity-comp-sb.model_activation.dataste_model_activation_timewindow_30D_V-1-5-0"

client = bigquery.Client(project=project_id)
query = f"SELECT * FROM `{table_fqn}`"

data_scoring = client.query(query).to_dataframe()
print(f"Datos cargados: {data_scoring.shape}")


Datos cargados: (2384265, 37)


In [57]:
# Filtrar solo activados (como en training)
if "label_activated_30d" in data_scoring.columns:
    n_before = len(data_scoring)
    data_scoring = data_scoring[data_scoring["label_activated_30d"] == 1].copy()
    print(f"Filtro activados: {n_before} -> {len(data_scoring)}")

Filtro activados: 2384265 -> 1470563


In [58]:
# Aplicar FeatureBuilder
bunch = fb_loaded.transform(data_scoring)
X = bunch.X

print(f"Features generadas: {X.shape}")

Features generadas: (1470563, 64)


In [59]:
# Alinear con schema del modelo (maneja features faltantes/extras)
X_aligned = X.reindex(columns=schema_loaded, fill_value=0)
print(f"Features alineadas: {X_aligned.shape}")

# Convertir a numpy
X_np = X_aligned.values.astype("float32")
print(f"Array numpy: {X_np.shape}, dtype: {X_np.dtype}")

Features alineadas: (1470563, 64)
Array numpy: (1470563, 64), dtype: float32


In [60]:
# Predicción
proba = model_loaded.predict_proba(X_np)[:, 1]

print(f"Predicciones: {len(proba)}")
print(f"   - Min:  {proba.min():.4f}")
print(f"   - Max:  {proba.max():.4f}")
print(f"   - Mean: {proba.mean():.4f}")
print(f"   - Std:  {proba.std():.4f}")

Predicciones: 1470563
   - Min:  0.0006
   - Max:  0.9999
   - Mean: 0.6293
   - Std:  0.3849


In [61]:
#  Output
import pandas as pd

output = pd.DataFrame({"p_5tx_30": proba})

# Agregar user_id 
if "user_id" in data_scoring.columns:
    output.insert(0, "user_id", data_scoring["user_id"].values)

print(f"Output shape: {output.shape}")
print(f"\n Preview:")
output.head(10)

Output shape: (1470563, 2)

 Preview:


,user_id,p_5tx_30
0,cc4a09d9-304e-4858-b4ef-6e85f23e3043,0.9681
1,cc4c8e46-ccaf-4777-ac5a-0eb1bf7f221a,0.0748
2,cc4dd43c-ce7e-43cf-b0e2-b4d83222711e,0.9663
3,cc4e3d12-d3d3-48ad-bc27-bfb5149d77ca,0.9750
4,cc5228ba-1e92-4269-8853-1bd9e6855fc1,0.9833
5,cc53712b-fb8d-498a-bf44-da419d6d9665,0.9996
6,cc53a785-a29f-4658-abab-6c7c0d0eed48,0.9802
7,cc54a268-19f6-403d-a725-0811513f26ff,0.9726
8,cc558fdb-0b7b-4f57-ae70-7edaf7d8273e,0.9983
9,cc55bbba-4ab4-4f28-9f8d-5450b3ea9f26,0.9976


In [62]:
# 1. Verificar varianza en las features
print("=" * 60)
print("DIAGNÓSTICO DE FEATURES")
print("=" * 60)

# Varianza por columna
print(f"\nShape de X_aligned: {X_aligned.shape}")
print(f"Shape de X (antes de align): {X.shape}")

# ¿Cuántas columnas tienen varianza > 0?
variances = X_aligned.var()
cols_with_variance = (variances > 0).sum()
cols_all_zero = (variances == 0).sum()

print(f"\nColumnas con varianza > 0: {cols_with_variance}")
print(f"Columnas con varianza = 0 (constantes): {cols_all_zero}")

# 2. ¿Cuántas columnas del FB coinciden con el schema?
fb_cols = set(X.columns)
schema_cols = set(schema_loaded)

matching = fb_cols & schema_cols
missing_in_fb = schema_cols - fb_cols
extra_in_fb = fb_cols - schema_cols

print(f"\n📊 Comparación FB vs Schema:")
print(f"   - Columnas que coinciden: {len(matching)}")
print(f"   - En schema pero NO en FB: {len(missing_in_fb)}")
print(f"   - En FB pero NO en schema: {len(extra_in_fb)}")

if len(missing_in_fb) > 0:
    print(f"\nColumnas faltantes (se llenaron con 0):")
    print(f"   {list(missing_in_fb)[:10]}...")  # Primeras 10

# 3. Preview de los datos
print(f"\nPreview de X_aligned (primeras 5 filas, 10 cols):")
print(X_aligned.iloc[:5, :10])

# 4. ¿Todo es cero?
total_zeros = (X_aligned == 0).sum().sum()
total_cells = X_aligned.shape[0] * X_aligned.shape[1]
pct_zeros = total_zeros / total_cells * 100
print(f"\n% de celdas que son cero: {pct_zeros:.2f}%")

DIAGNÓSTICO DE FEATURES

Shape de X_aligned: (1470563, 64)
Shape de X (antes de align): (1470563, 64)

Columnas con varianza > 0: 61
Columnas con varianza = 0 (constantes): 3

📊 Comparación FB vs Schema:
   - Columnas que coinciden: 64
   - En schema pero NO en FB: 0
   - En FB pero NO en schema: 0

📋 Preview de X_aligned (primeras 5 filas, 10 cols):
   userTypeIdentifier  accountLevel  IsActive  phn_confir  email_confir  \
0              3.0000        2.0000         1      0.0000        0.0000   
1              3.0000        2.0000         1      0.0000        0.0000   
2              3.0000        2.0000         1      0.0000        0.0000   
3              3.0000        2.0000         1      0.0000        0.0000   
4              3.0000        2.0000         1      0.0000        0.0000   

   phn_confir_d7  email_confir_d7  both_confir_d7  has_premia  \
0         1.0000           0.0000          0.0000      1.0000   
1         1.0000           1.0000          1.0000      1.0000   
2

In [63]:
# Ver qué tiene cada uno
print("Schema guardado (primeras 10):")
print(schema_loaded[:10])

print("\nColumnas del FeatureBuilder (primeras 10):")
print(list(X.columns)[:10])

Schema guardado (primeras 10):
['userTypeIdentifier', 'accountLevel', 'IsActive', 'phn_confir', 'email_confir', 'phn_confir_d7', 'email_confir_d7', 'both_confir_d7', 'has_premia', 'label_activated_30d']

Columnas del FeatureBuilder (primeras 10):
['userTypeIdentifier', 'accountLevel', 'IsActive', 'phn_confir', 'email_confir', 'phn_confir_d7', 'email_confir_d7', 'both_confir_d7', 'has_premia', 'label_activated_30d']
